# 实验 5.3：后门检测（Backdoor Detection）

## 🎯 学习目标（Learning Objectives）
完成本实验后，你将能够：
- ✅ 用自己的话解释“后门检测为什么难”
- ✅ 提取神经网络中间层激活（activation）并进行对比分析
- ✅ 使用 PCA 可视化激活分布，观察后门导致的异常模式
- ✅ 实现一个简单的“激活敏感度”后门检测器，并解读检测分数

## 📚 前置知识
- 完成实验 5.2：BadNets 后门攻击
- 理解分类任务的基本流程（训练 / 测试）
- 了解“中间层激活”的含义（大概知道即可）

## 🖥️ 实验环境
- 平台：腾讯 Cloud Studio（https://cloudstudio.net/）
- GPU：NVIDIA Tesla T4（16GB）
- 模型：自定义 PyTorch 分类网络（本实验内训练）

## 📝 填空说明
本实验共 **5 个填空**，难度：⭐⭐⭐☆☆  
请将代码中的 `___________` 替换为正确代码后运行。

⚠️ 安全提醒：本实验仅用于教育目的，帮助理解后门检测技术，请勿用于未授权场景。

## 📖 概念回顾

### 后门检测的挑战

为什么后门检测很难？

| 挑战 | 说明 |
|------|------|
| 触发器未知 | 不知道什么会激活后门 |
| 正常测试通过 | 后门模型在干净数据上表现正常 |
| 多样性 | 触发器形式多样（图案、位置、大小） |

### 检测思路

核心思想：**后门模型的内部行为与正常模型不同**

```
正常模型：  输入 A → 激活模式 X → 输出
            输入 B → 激活模式 Y → 输出

后门模型：  正常输入 → 正常激活 → 正常输出
            触发输入 → 异常激活 → 目标输出
                       ^^^^^^^^
                       检测重点！
```

### 激活聚类方法

通过分析模型中间层的激活值，寻找异常模式：
- 提取模型对不同输入的激活值
- 对激活值进行聚类/降维分析
- 后门输入的激活往往与正常输入有明显分离

## 第一部分：环境准备

In [ ]:
# ====== 环境准备与验证 ======
# 安装依赖（首次运行需要一点时间）
%pip install torch numpy matplotlib -q

import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Matplotlib 中文显示（不同环境下可能使用不同字体）
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 固定随机种子，保证结果更稳定
torch.manual_seed(42)
np.random.seed(42)

print("-" * 50)
print("🔍 后门检测实验环境检查")
print("-" * 50)

print(f"\n[1] Python 版本: {sys.version.split()[0]}")
print(f"[2] PyTorch 版本: {torch.__version__}")
print(f"    CUDA 可用: {'✓ 是' if torch.cuda.is_available() else '✗ 否'}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"    GPU 型号: {gpu_name}")
    print(f"    GPU 显存: {gpu_memory:.1f} GB")

print("\n" + "-" * 50)
print("✅ 环境检查通过！" if torch.cuda.is_available() else "⚠️ 未检测到 GPU，将使用 CPU 运行")
print("-" * 50)

## 第二部分：准备数据和模型

复用实验 5.2 的代码，创建干净模型和后门模型进行对比。

In [ ]:
# ============ 数据生成函数 ============

def create_image_dataset(n_samples=500):
    """
    创建简化的图像分类数据集
    类别 0：左上角亮点
    类别 1：右下角亮点
    """
    images = []
    labels = []
    
    for i in range(n_samples):
        img = np.random.rand(8, 8) * 0.3
        if i < n_samples // 2:
            img[0:2, 0:2] += 0.7
            labels.append(0)
        else:
            img[6:8, 6:8] += 0.7
            labels.append(1)
        img = np.clip(img, 0, 1)
        images.append(img)
    
    indices = np.random.permutation(n_samples)
    images = [images[i] for i in indices]
    labels = [labels[i] for i in indices]
    return np.array(images), np.array(labels)


def add_trigger(image, trigger_value=1.0):
    """在图像中心添加 2x2 触发器"""
    triggered_image = image.copy()
    triggered_image[3:5, 3:5] = trigger_value
    return triggered_image


def create_poisoned_dataset(X, y, poison_ratio=0.1, target_label=0):
    """创建后门投毒数据集"""
    X_poisoned = X.copy()
    y_poisoned = y.copy()
    n_samples = len(X)
    n_poison = int(n_samples * poison_ratio)
    poison_indices = np.random.choice(n_samples, n_poison, replace=False)
    
    for idx in poison_indices:
        X_poisoned[idx] = add_trigger(X_poisoned[idx])
        y_poisoned[idx] = target_label
    
    return X_poisoned, y_poisoned, poison_indices

# 创建数据
X_train, y_train = create_image_dataset(400)
X_test, y_test = create_image_dataset(100)

# 创建投毒数据（10% 投毒，目标类别 0）
X_train_poisoned, y_train_poisoned, _ = create_poisoned_dataset(
    X_train, y_train, poison_ratio=0.1, target_label=0
)

print("=" * 50)
print("📊 数据准备完成")
print("=" * 50)
print(f"  训练集: {len(X_train)} 张图像")
print(f"  测试集: {len(X_test)} 张图像")
print(f"  投毒比例: 10%")

In [ ]:
# ============ 模型定义（带激活提取功能）============

class SimpleCNNWithActivation(nn.Module):
    """
    可以提取中间层激活值的分类网络
    
    结构：展平(64) → fc1(32) → fc2(16) → 输出(2)
                              ^^^^^^^^
                              我们提取这层的激活值
    """
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        self.activation = None  # 用于存储中间层激活
    
    def forward(self, x):
        x = x.view(-1, 64)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        self.activation = x.detach()  # 保存 fc2 后的激活值
        x = self.fc3(x)
        return x
    
    def get_activation(self, x):
        """获取中间层激活值"""
        _ = self.forward(x)  # 运行前向传播
        return self.activation


def train_model(X_train, y_train, epochs=150):
    """训练模型"""
    X = torch.FloatTensor(X_train)
    y = torch.LongTensor(y_train)
    
    model = SimpleCNNWithActivation()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    
    return model

# 训练两个模型
print("\n🤖 训练模型...")
print("  训练干净模型...")
clean_model = train_model(X_train, y_train)
print("  ✓ 干净模型训练完成")

print("  训练后门模型...")
backdoor_model = train_model(X_train_poisoned, y_train_poisoned)
print("  ✓ 后门模型训练完成")

### ✅ 检查点 1

运行到这里，你应该看到：
- [ ] 数据准备完成（400 张训练图像，100 张测试图像）
- [ ] 干净模型和后门模型都训练完成

## 第三部分：提取激活值

激活聚类的核心：**分析模型对不同输入的内部表示（激活值）**。

后门模型的特点是：
- 对正常输入：正常的激活模式
- 对触发输入：异常的激活模式（与正常明显不同）

In [ ]:
def extract_activations(model, X):
    """
    提取模型对输入的中间层激活值
    
    参数:
        model: 训练好的模型
        X: 输入数据
    
    返回:
        activations: 激活值数组 (n_samples, 16)
    """
    X_tensor = torch.FloatTensor(X)
    
    # ========== 填空 1：获取中间层激活值 ==========
    # 
    # 🎯 任务：使用模型的 get_activation 方法获取激活值
    # 
    # 💡 提示：
    #   - 调用 model.get_activation(X_tensor)
    #   - 使用 .numpy() 转换为 NumPy 数组
    # 
    # 难度：⭐☆☆☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    activations = ___________  # 期望：model.get_activation(X_tensor).numpy()
    
    # ========== 填空 1 结束 ==========
    
    return activations

# 准备测试数据：干净样本和触发样本
X_clean = X_test.copy()
X_triggered = np.array([add_trigger(img) for img in X_test])

print("=" * 50)
print("📊 测试数据准备")
print("=" * 50)
print(f"  干净样本数量: {len(X_clean)}")
print(f"  触发样本数量: {len(X_triggered)}")

## 第四部分：分析激活差异

比较两种模型对干净输入和触发输入的激活差异。

In [ ]:
# 提取四种情况的激活值
act_clean_on_clean = extract_activations(clean_model, X_clean)
act_clean_on_triggered = extract_activations(clean_model, X_triggered)
act_backdoor_on_clean = extract_activations(backdoor_model, X_clean)
act_backdoor_on_triggered = extract_activations(backdoor_model, X_triggered)

print("=" * 50)
print("📊 激活值提取完成")
print("=" * 50)
print(f"  激活值维度: {act_clean_on_clean.shape}")
print(f"  每个样本有 {act_clean_on_clean.shape[1]} 个神经元的激活值")


def compute_activation_difference(act1, act2):
    """
    计算两组激活值的平均差异
    
    参数:
        act1: 第一组激活值的平均向量
        act2: 第二组激活值的平均向量
    
    返回:
        diff: 平均绝对差异
    """
    # ========== 填空 2：计算激活差异 ==========
    # 
    # 🎯 任务：计算两组激活值的平均绝对差异
    # 
    # 💡 提示：
    #   - 使用 np.abs() 计算绝对值
    #   - 使用 np.mean() 计算平均值
    #   - 公式：mean(|act1 - act2|)
    # 
    # 难度：⭐⭐☆☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    diff = ___________  # 期望：np.mean(np.abs(act1 - act2))
    
    # ========== 填空 2 结束 ==========
    
    return diff

# 计算激活差异
diff_clean_model = compute_activation_difference(
    act_clean_on_clean.mean(axis=0), 
    act_clean_on_triggered.mean(axis=0)
)

diff_backdoor_model = compute_activation_difference(
    act_backdoor_on_clean.mean(axis=0), 
    act_backdoor_on_triggered.mean(axis=0)
)

print("\n" + "=" * 50)
print("🔍 激活差异分析")
print("=" * 50)
print(f"\n  干净模型：干净输入 vs 触发输入")
print(f"    激活差异: {diff_clean_model:.4f}")
print(f"\n  后门模型：干净输入 vs 触发输入")
print(f"    激活差异: {diff_backdoor_model:.4f}")
print(f"\n  差异比值: {diff_backdoor_model/diff_clean_model:.2f}x")

if diff_backdoor_model > diff_clean_model * 1.5:
    print("\n  ⚠️ 后门模型的激活差异明显更大，可能存在后门！")
else:
    print("\n  ✓ 两个模型的激活差异相近")

### 🤔 思考一下

1. **观察**：后门模型的激活差异是干净模型的多少倍？
2. **分析**：为什么后门模型对触发器的"反应"更大？
3. **应用**：这种差异可以用来检测后门吗？

## 第五部分：可视化激活分布

使用 PCA 降维将高维激活值可视化到 2D 空间，观察后门导致的分布分离。

In [ ]:
def simple_pca_2d(data):
    """
    简化的 PCA 降维到 2 维
    
    参数:
        data: 输入数据 (n_samples, n_features)
    
    返回:
        projected: 降维后的数据 (n_samples, 2)
    """
    # 中心化
    data_centered = data - data.mean(axis=0)
    
    # 计算协方差矩阵
    cov = np.cov(data_centered.T)
    
    # 特征分解
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    
    # 选择前两个主成分（最大特征值对应的特征向量）
    idx = eigenvalues.argsort()[::-1]
    top2_eigenvectors = eigenvectors[:, idx[:2]]
    
    # 投影
    projected = data_centered @ top2_eigenvectors
    return projected

# 可视化两个模型的激活分布
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ===== 干净模型的激活分布 =====
all_act_clean = np.vstack([act_clean_on_clean, act_clean_on_triggered])
pca_clean = simple_pca_2d(all_act_clean)
n = len(X_clean)

axes[0].scatter(pca_clean[:n, 0], pca_clean[:n, 1],
                c='blue', alpha=0.6, s=40, label='干净输入')
axes[0].scatter(pca_clean[n:, 0], pca_clean[n:, 1],
                c='red', alpha=0.6, s=40, label='触发输入')
axes[0].set_title('干净模型的激活分布', fontsize=14)
axes[0].set_xlabel('主成分 1', fontsize=12)
axes[0].set_ylabel('主成分 2', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# ===== 后门模型的激活分布 =====
all_act_backdoor = np.vstack([act_backdoor_on_clean, act_backdoor_on_triggered])
pca_backdoor = simple_pca_2d(all_act_backdoor)

axes[1].scatter(pca_backdoor[:n, 0], pca_backdoor[:n, 1],
                c='blue', alpha=0.6, s=40, label='干净输入')
axes[1].scatter(pca_backdoor[n:, 0], pca_backdoor[n:, 1],
                c='red', alpha=0.6, s=40, label='触发输入')
axes[1].set_title('后门模型的激活分布', fontsize=14)
axes[1].set_xlabel('主成分 1', fontsize=12)
axes[1].set_ylabel('主成分 2', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.suptitle('激活聚类分析：寻找后门导致的异常模式', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\n📌 观察要点：")
print("  - 左图（干净模型）：干净输入和触发输入的激活分布混合在一起")
print("  - 右图（后门模型）：触发输入的激活分布与干净输入明显分离")
print("  - 这种分离是后门的‘指纹’，可用于检测！")

### ✅ 检查点 2

运行到这里，你应该观察到：
- [ ] 左图：蓝色和红色点混合在一起（干净模型无分离）
- [ ] 右图：红色点明显偏离蓝色点（后门模型有分离）

这种分离是后门检测的关键信号！

## 第六部分：实现简单的后门检测器

基于激活分析的检测思路：
1. 对输入添加可能的扰动（模拟触发器）
2. 比较扰动前后的激活变化
3. 后门模型的激活变化更剧烈

In [ ]:
def detect_backdoor(model, X_test, threshold=0.1):
    """
    简单的后门检测方法
    
    思路：比较模型对原始输入和添加扰动后输入的激活差异
    后门模型对触发器位置的扰动更敏感
    
    参数:
        model: 待检测的模型
        X_test: 测试数据
        threshold: 检测阈值
    
    返回:
        is_backdoor: 是否可能有后门
        score: 检测分数（激活变化程度）
    """
    # 获取原始激活
    act_original = extract_activations(model, X_test)
    
    # 添加随机扰动（模拟可能的触发器位置）
    X_perturbed = X_test.copy()
    for i in range(len(X_perturbed)):
        # 在中间位置添加扰动（与触发器位置相同）
        X_perturbed[i, 3:5, 3:5] = np.random.rand(2, 2)
    
    # 获取扰动后的激活
    act_perturbed = extract_activations(model, X_perturbed)
    
    # ========== 填空 3：计算激活变化的标准差 ==========
    # 
    # 🎯 任务：计算每个样本激活变化的范数，然后求标准差
    # 
    # 💡 提示：
    #   - 使用 np.linalg.norm(..., axis=1) 计算每行的范数
    #   - 使用 np.std() 计算标准差
    #   - 步骤：1) 计算差值 2) 计算每行范数 3) 求标准差
    # 
    # 难度：⭐⭐⭐☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    activation_change = ___________  # 期望：np.std(np.linalg.norm(act_original - act_perturbed, axis=1))
    
    # ========== 填空 3 结束 ==========
    
    # 判断是否存在后门
    is_backdoor = activation_change > threshold
    
    return is_backdoor, activation_change

# 检测两个模型
print("=" * 60)
print("🔍 后门检测结果")
print("=" * 60)

is_backdoor_clean, score_clean = detect_backdoor(clean_model, X_test)
is_backdoor_back, score_back = detect_backdoor(backdoor_model, X_test)

print(f"\n  干净模型:")
print(f"    激活变化分数: {score_clean:.4f}")
status_clean = '⚠️ 可能有后门' if is_backdoor_clean else '✓ 正常'
print(f"    检测结果: {status_clean}")

print(f"\n  后门模型:")
print(f"    激活变化分数: {score_back:.4f}")
status_back = '⚠️ 可能有后门' if is_backdoor_back else '✓ 正常'
print(f"    检测结果: {status_back}")

print(f"\n  分数比值: {score_back/score_clean:.2f}x")
print("\n" + "=" * 60)

## 第七部分：神经元级别分析

分析每个神经元在干净输入和触发输入上的平均激活，寻找"后门神经元"。

In [ ]:
def analyze_neuron_activation(model, X_clean, X_triggered):
    """
    分析每个神经元在干净输入和触发输入上的平均激活

    参数:
        model: 训练好的模型
        X_clean: 干净输入
        X_triggered: 触发输入

    返回:
        mean_clean: 干净输入的平均激活 (16,)
        mean_triggered: 触发输入的平均激活 (16,)
    """
    act_clean = extract_activations(model, X_clean)
    act_triggered = extract_activations(model, X_triggered)

    # ========== 填空 4：计算平均激活 ==========
    #
    # 🎯 任务：计算每个神经元的平均激活值
    #
    # 💡 提示：
    #   - 使用 .mean(axis=0) 沿样本维度求平均
    #   - 结果应该是长度为 16 的向量（16 个神经元）
    #
    # 难度：⭐⭐☆☆☆
    #
    # 请将 ___________ 替换为正确的代码

    mean_clean = ___________      # 期望：act_clean.mean(axis=0)
    mean_triggered = ___________  # 期望：act_triggered.mean(axis=0)

    # ========== 填空 4 结束 ==========

    return mean_clean, mean_triggered


# 获取两个模型的神经元激活
clean_act_c, clean_act_t = analyze_neuron_activation(clean_model, X_clean, X_triggered)
back_act_c, back_act_t = analyze_neuron_activation(backdoor_model, X_clean, X_triggered)

# 可视化神经元激活对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(len(clean_act_c))  # 神经元编号
width = 0.35

# 干净模型
axes[0].bar(x - width/2, clean_act_c, width, label='干净输入', color='blue', alpha=0.7)
axes[0].bar(x + width/2, clean_act_t, width, label='触发输入', color='red', alpha=0.7)
axes[0].set_xlabel('神经元编号', fontsize=12)
axes[0].set_ylabel('平均激活值', fontsize=12)
axes[0].set_title('干净模型：各神经元激活对比', fontsize=14)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_xticks(x)

# 后门模型
axes[1].bar(x - width/2, back_act_c, width, label='干净输入', color='blue', alpha=0.7)
axes[1].bar(x + width/2, back_act_t, width, label='触发输入', color='red', alpha=0.7)
axes[1].set_xlabel('神经元编号', fontsize=12)
axes[1].set_ylabel('平均激活值', fontsize=12)
axes[1].set_title('后门模型：各神经元激活对比', fontsize=14)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_xticks(x)

plt.suptitle('神经元激活分析：寻找“后门神经元”', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# 找出激活差异最大的神经元

diff_clean = np.abs(clean_act_t - clean_act_c)
diff_back = np.abs(back_act_t - back_act_c)

print("\n📊 神经元激活差异分析：")
print(f"\n  干净模型：最大差异神经元 #{np.argmax(diff_clean)}，差异 = {diff_clean.max():.4f}")
print(f"  后门模型：最大差异神经元 #{np.argmax(diff_back)}，差异 = {diff_back.max():.4f}")
print("\n  后门模型中差异较大的神经元可能是‘后门神经元’")

## 第八部分：检测方法总结

In [ ]:
# ========== 填空 5：创建检测方法对比表 ==========
# 
# 🎯 任务：定义常见后门检测方法的信息
# 
# 💡 提示：
#   - 每个方法包含：(名称, 检测时机, 需要信息, 计算成本, 主要优势)
#   - 常见方法：激活聚类、Neural Cleanse、STRIP
# 
# 难度：⭐⭐☆☆☆

detection_methods = [
    # (方法名, 检测时机, 需要的信息, 计算成本, 主要优势)
    ("激活聚类", "训练后", "训练数据", "中", "简单直观"),
    ("Neural Cleanse", "训练后", "模型访问", "高", "可重建触发器"),
    # 请填写 STRIP 方法的信息
    ("STRIP", ___________, ___________, ___________, ___________)  # 期望："运行时", "输入样本", "低", "实时检测"
]

# ========== 填空 5 结束 ==========

print("=" * 75)
print("📋 后门检测方法对比")
print("=" * 75)
print(f"{'方法':<18} {'检测时机':<12} {'需要的信息':<14} {'计算成本':<10} {'主要优势'}")
print("-" * 75)
for m in detection_methods:
    print(f"{m[0]:<18} {m[1]:<12} {m[2]:<14} {m[3]:<10} {m[4]}")
print("=" * 75)

print("\n📝 方法说明：")
print("  - 激活聚类：分析模型中间层激活，寻找异常分布（本实验方法）")
print("  - Neural Cleanse：优化求解最小触发器，检测并重建后门")
print("  - STRIP：运行时检测，通过叠加扰动观察输出变化")

## 📋 实验小结

### 核心收获

1. **概念**：理解了后门检测的核心挑战——触发器未知、正常测试通过

2. **技能**：掌握了基于激活分析的后门检测方法——通过观察模型内部行为发现异常

3. **思考**：认识到检测与攻击是持续对抗的过程，没有完美的检测方法

### 关键代码回顾

```python
# 激活分析的核心
activations = model.get_activation(X_tensor).numpy()  # 提取激活
diff = np.mean(np.abs(act1 - act2))                   # 计算差异
```

### 检测思路总结

| 检测信号 | 正常模型 | 后门模型 |
|---------|---------|----------|
| 激活差异 | 小 | 大 |
| PCA 分布 | 混合 | 分离 |
| 神经元响应 | 一致 | 异常神经元 |

### 防御建议

1. **部署前审计**：使用 Neural Cleanse 等工具检测
2. **运行时监控**：使用 STRIP 检测可疑输入
3. **多方法组合**：高安全场景使用多种检测方法
4. **供应链安全**：验证模型来源，使用安全格式（如 SafeTensors）

### 延伸思考

1. 如果攻击者知道你使用激活聚类检测，他能如何改进攻击？
2. 除了检测，还有什么方法可以移除已植入的后门？

## 📚 参考答案

<details>
<summary>点击展开填空参考答案</summary>

**填空 1：获取中间层激活值**
```python
activations = model.get_activation(X_tensor).numpy()
```

**填空 2：计算激活差异**
```python
diff = np.mean(np.abs(act1 - act2))
```

**填空 3：计算激活变化的标准差**
```python
activation_change = np.std(np.linalg.norm(act_original - act_perturbed, axis=1))
```

**填空 4：计算平均激活**
```python
mean_clean = act_clean.mean(axis=0)
mean_triggered = act_triggered.mean(axis=0)
```

**填空 5：STRIP 方法信息**
```python
("STRIP", "运行时", "输入样本", "低", "实时检测")
```

</details>

## 实验结束：释放资源

完成实验后建议释放内存和显存，避免占用 GPU 额度（清理代码见下一单元）。


In [ ]:
# 实验结束，清理资源（可选）
# 如果你要继续进行下一个实验，建议先运行这段代码释放内存/显存

import gc

# 删除大对象（如果存在）
for var_name in [
    'clean_model',
    'backdoor_model',
    'act_clean_on_clean',
    'act_clean_on_triggered',
    'act_backdoor_on_clean',
    'act_backdoor_on_triggered',
]:
    if var_name in globals():
        del globals()[var_name]

# Python 垃圾回收
gc.collect()

# 清理 CUDA 显存
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("✓ 资源已清理")

⚠️ 重要提醒

实验完成后，请记得：

1. **释放 GPU 资源**：如果你使用的是 Tencent CloudStudio, 请点击右上角的 "停止" 按钮，停止运行。
2. **保存实验结果**：如需保存，请下载 notebook 文件

这样可以避免不必要的资源占用和费用产生。**否则会一直消耗你的免费资源额度。**